In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt 
import datetime
from sklearn.metrics import fbeta_score
import lightgbm as lgb

In [3]:
class Config:
    pass
config = pd.read_pickle('config.pkl')
data_path = config.data_path
feature_path = config.feature_path
print(dir(config))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'data_path', 'feature_dict', 'feature_path', 'model_features', 'result_path', 'single_module_validation_indice_set', 'trade_train_size', 'train_2_6_index']


In [4]:
#model define

def f_beta_01(preds, train_data):
    labels  = train_data.get_label()
    return 'fbeta_score_01',fbeta_score(labels, preds > 0.5,0.1),True

    
#for binary
def runLGBM(train_X, train_y, test_X, test_y=None, feature_names=None,
           seed_val=0, num_rounds=10000,watch_dict = None,max_bin=50000,
           num_leaves=16,early_stop=64,verbose=True,eta=0.1,
           bagging_fraction = 0.75 , feature_fraction = 0.75,feval = None,metric = 'binary_logloss',
           train_sample_weight = None):
    
    params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': num_leaves,
        'learning_rate': eta,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': 5,
        'verbose': verbose,
        'is_unbalance':False
    }
    
    num_rounds = num_rounds

    #plst = list(param.items())
    lgbtrain = lgb.Dataset(train_X, label=train_y,max_bin=max_bin,feature_name=feature_names,weight =train_sample_weight)

    if test_y is not None:
        lgbtest = lgb.Dataset(test_X, label=test_y,max_bin=max_bin,feature_name=feature_names)
        watchlist = [lgbtrain,lgbtest]
        watchlist_name=['train','test']
        model = lgb.train(params, lgbtrain, num_rounds, watchlist,watchlist_name, early_stopping_rounds=early_stop,\
                         evals_result = watch_dict,verbose_eval=verbose,feval = feval)
    else:
        #lgbtest = lgb.Dataset(test_X,feature_name=feature_names)
        model = lgb.train(params, lgbtrain, num_rounds)

    pred_test_y = model.predict(test_X)
    return pred_test_y, model

In [6]:
features =   (config.feature_dict['trade_detail_feature']+
              config.feature_dict['recent_login_detail']+
              config.feature_dict['trade_and_recent_login_comparing']+
              config.feature_dict['login_trade_hist_stats'])
feature_sequence_list = []
for feature in features:
    feature_sequence_list.append(pd.read_pickle(feature_path+feature+'.pkl').reshape(-1,1))
    
trade_tt_mat = np.hstack(feature_sequence_list)
#trade_tt_mat[trade_tt_mat==-10]=np.nan

validation_tuple_list = config.single_module_validation_indice_set
train_labels = pd.read_pickle(data_path+'trade_train_label.pkl')

In [8]:
cv_scores = []
cv_result = []
models = []
i = 0

for train_indice,val_indice in validation_tuple_list:
    #print trade_train_val.iloc[train_indice]['month'].unique(),trade_train_val.iloc[val_indice]['month'].unique()
    #print trade_train_val.iloc[train_indice].shape,trade_train_val.iloc[val_indice].shape
    result_dict = {}
    
    #filter the features
    dev_X, val_X = trade_tt_mat[train_indice], trade_tt_mat[val_indice]
    dev_y, val_y = train_labels.iloc[train_indice].values, train_labels.iloc[val_indice].values

    
    preds, model = runLGBM(dev_X, dev_y, val_X, val_y,feature_names=features,verbose=100,eta=0.02,
                          early_stop=None,num_rounds=600,watch_dict=result_dict,feval = f_beta_01)

    #result_f_beta = f_beta_01(val_y.values, preds>0.5)
    result_f_beta  = fbeta_score( val_y,preds > 0.5, 0.1)
    
    cv_scores.append(result_f_beta)
    cv_result.append(result_dict)
    models.append(model)
    i+=1
    print('f_beta score for the turn '+str(i)+' is '+str(result_f_beta))

print('The mean of the cv_scores is:',np.mean(cv_scores))

/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


[100]	train's auc: 0.93919	train's fbeta_score_01: 0.923553	test's auc: 0.922661	test's fbeta_score_01: 0.831276
[200]	train's auc: 0.97702	train's fbeta_score_01: 0.937239	test's auc: 0.959978	test's fbeta_score_01: 0.833181
[300]	train's auc: 0.992494	train's fbeta_score_01: 0.942157	test's auc: 0.967505	test's fbeta_score_01: 0.823244
[400]	train's auc: 0.997036	train's fbeta_score_01: 0.954429	test's auc: 0.967873	test's fbeta_score_01: 0.823529
[500]	train's auc: 0.998632	train's fbeta_score_01: 0.969252	test's auc: 0.967099	test's fbeta_score_01: 0.821403
[600]	train's auc: 0.999521	train's fbeta_score_01: 0.983373	test's auc: 0.966179	test's fbeta_score_01: 0.798706
f_beta score for the turn 1 is 0.79870596693
[100]	train's auc: 0.95801	train's fbeta_score_01: 0.932748	test's auc: 0.846613	test's fbeta_score_01: 0.118475
[200]	train's auc: 0.983573	train's fbeta_score_01: 0.943383	test's auc: 0.886538	test's fbeta_score_01: 0.352618
[300]	train's auc: 0.99384	train's fbeta_score

In [9]:
#finding the best iteration
pd_list = []
for dic in cv_result:
    pd_list.append(pd.DataFrame(dic['test']))
for i in range(len(pd_list)):
    pd_list[i].columns = pd_list[i].columns+'_'+str(i)
validation_result = pd.concat(pd_list,axis = 1)
validation_result['auc_avg'] = validation_result.apply(lambda x : np.mean([x.auc_0,x.auc_1,x.auc_2,x.auc_3,x.auc_4]),axis = 1)
validation_result['fbeta_avg'] = validation_result.apply(lambda x : np.mean([x.fbeta_score_01_0,x.fbeta_score_01_1,
                                                                     x.fbeta_score_01_2,x.fbeta_score_01_3,
                                                                     x.fbeta_score_01_4]),axis=1)

In [11]:
print(validation_result['auc_avg'].idxmax())
print(validation_result['fbeta_avg'].idxmax())

488
288


In [12]:
train_X = trade_tt_mat[config.train_2_6_index]
test_X = trade_tt_mat[config.trade_train_size:]
train_y = train_labels[config.train_2_6_index]

preds, _ = runLGBM(train_X, train_y, test_X, feature_names=features,verbose=100,eta=0.02,
                          early_stop=None,num_rounds=488,watch_dict=None)


/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


In [15]:
result_path = '../../kaggleData/JD_logging/result/'
test_rowkey = pd.read_pickle(data_path+'trade_test_rowkey.pkl')
pred_label = pd.Series(preds > 0.5)
result_set = pd.DataFrame(test_rowkey)
result_set['is_risk'] = pred_label.astype(int)

print(result_set.shape)

(17875, 2)


In [16]:
pd.to_pickle(pred_label,result_path+'adding_type_C_488.pkl')
result_set.to_csv(result_path+'adding_type_C_488.csv',index=False,header=False)

In [13]:
train_X = trade_tt_mat[config.train_2_6_index]
test_X = trade_tt_mat[config.trade_train_size:]
train_y = train_labels[config.train_2_6_index]

preds, _ = runLGBM(train_X, train_y, test_X, feature_names=features,verbose=100,eta=0.02,
                          early_stop=None,num_rounds=288,watch_dict=None)

result_path = '../../kaggleData/JD_logging/result/'
test_rowkey = pd.read_pickle(data_path+'trade_test_rowkey.pkl')
pred_label = pd.Series(preds > 0.5)
result_set = pd.DataFrame(test_rowkey)
result_set['is_risk'] = pred_label.astype(int)

print result_set.shape

pd.to_pickle(pred_label,result_path+'adding_type_C_482turn.pkl')
result_set.to_csv(result_path+'adding_type_C_482turn.csv',index=False,header=False)

(17875, 2)
